In [ ]:
# connecting drive to colab notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# installing requirements
!pip install imageio==2.19.3
!pip install opencv-python==4.6.0.66
!pip install tensorboardX==2.2
!pip install einops==0.4.1
!pip install scikit-learn==1.1.1
!pip install matplotlib==3.5.1
!pip install protobuf==3.20.0
!pip install fire==0.4.0
!pip install moviepy==1.0.3

In [54]:
import sys
sys.path.append('/content/drive/My Drive/drive_folder/pips/')
import time
import numpy as np
import io
import os
from PIL import Image
import cv2
import saverloader
import imageio as imageio
from nets.pips import Pips
import utils.improc
import random
import glob
from utils.basic import print_, print_stats
import torch
from tensorboardX import SummaryWriter
import torch.nn.functional as F
import matplotlib.pyplot as plt

# save numpy array as npz file
from numpy import asarray
from numpy import savez_compressed

In [4]:
def create_model(model_weights):
  model = Pips(stride=4).cuda()
  parameters = list(model.parameters())
  _ = saverloader.load(model_weights, model)
  global_step = 0
  model.eval()
  return model

In [5]:
# random.seed(125)
# np.random.seed(125)

def run_model(model, rgbs, N):
    rgbs = rgbs.cuda().float() # B, S, C, H, W

    B, S, C, H, W = rgbs.shape
    rgbs_ = rgbs.reshape(B*S, C, H, W)
    H_, W_ = 360, 640
    rgbs_ = F.interpolate(rgbs_, (H_, W_), mode='bilinear')
    H, W = H_, W_
    rgbs = rgbs_.reshape(B, S, C, H, W)

    # pick N points to track; we'll use a uniform grid
    N_ = np.sqrt(N).round().astype(np.int32)
    grid_y, grid_x = utils.basic.meshgrid2d(B, N_, N_, stack=False, norm=False, device='cuda')
    grid_y = 8 + grid_y.reshape(B, -1)/float(N_-1) * (H-16)
    grid_x = 8 + grid_x.reshape(B, -1)/float(N_-1) * (W-16)
    xy = torch.stack([grid_x, grid_y], dim=-1) # B, N_*N_, 2
    _, S, C, H, W = rgbs.shape

    print_stats('rgbs', rgbs)
    preds, preds_anim, vis_e, stats = model(xy, rgbs, iters=6)
    trajs_e = preds[-1]
    print_stats('trajs_e', trajs_e)
    
    pad = 50
    rgbs = F.pad(rgbs.reshape(B*S, 3, H, W), (pad, pad, pad, pad), 'constant', 0).reshape(B, S, 3, H+pad*2, W+pad*2)
    trajs_e = trajs_e + pad
    
    # if sw is not None and sw.save_this:
    #     linewidth = 2

    #     # visualize the input
    #     o1 = sw.summ_rgbs('inputs/rgbs', utils.improc.preprocess_color(rgbs[0:1]).unbind(1))
    #     # visualize the trajs overlaid on the rgbs
    #     o2 = sw.summ_traj2ds_on_rgbs('outputs/trajs_on_rgbs', trajs_e[0:1], utils.improc.preprocess_color(rgbs[0:1]), cmap='spring', linewidth=linewidth)
    #     # visualize the trajs alone
    #     o3 = sw.summ_traj2ds_on_rgbs('outputs/trajs_on_black', trajs_e[0:1], torch.ones_like(rgbs[0:1])*-0.5, cmap='spring', linewidth=linewidth)
    #     # concat these for a synced wide vis
    #     wide_cat = torch.cat([o1, o2, o3], dim=-1)
    #     sw.summ_rgbs('outputs/wide_cat', wide_cat.unbind(1))

    #     # write to disk, in case that's more convenient
    #     wide_list = list(wide_cat.unbind(1))
    #     wide_list = [wide[0].permute(1,2,0).cpu().numpy() for wide in wide_list]
    #     wide_list = [Image.fromarray(wide) for wide in wide_list]
    #     out_fn = './out_%d.gif' % sw.global_step
    #     wide_list[0].save(out_fn, save_all=True, append_images=wide_list[1:])
    #     print('saved %s' % out_fn)

    #     # alternate vis
    #     sw.summ_traj2ds_on_rgbs2('outputs/trajs_on_rgbs2', trajs_e[0:1], vis_e[0:1], utils.improc.preprocess_color(rgbs[0:1]))
        
    #     # animation of inference iterations
    #     rgb_vis = []
    #     for trajs_e_ in preds_anim:
    #         trajs_e_ = trajs_e_ + pad
    #         rgb_vis.append(sw.summ_traj2ds_on_rgb('', trajs_e_[0:1], torch.mean(utils.improc.preprocess_color(rgbs[0:1]), dim=1), cmap='spring', linewidth=linewidth, only_return=True))
    #     sw.summ_rgbs('outputs/animated_trajs_on_rgb', rgb_vis)

# return vis??
    return trajs_e-pad

In [79]:
def generate_training_data(pips_model, video_name):
    ## choose hyps
    B = 1
    S = 8
    N = 2**2 # number of points to track

    filenames = glob.glob('data_small/content/frames/'+video_name+'/frames/*_001*.jpg')
    filenames = sorted(filenames)
    max_iters = len(filenames)//S # run each unique subsequence

    model = pips_model

    global_step = 0
    print(global_step)
    print(max_iters)

    a = np.empty([1,2])

    while global_step < max_iters:
        
        global_step += 1

        try:
            rgbs = []
            for s in range(S):
                fn = filenames[(global_step-1)*S+s]
                if s==0:
                    print('start frame', fn)
                im = imageio.imread(fn)
                im = im.astype(np.uint8)
                rgbs.append(torch.from_numpy(im).permute(2,0,1))
            rgbs = torch.stack(rgbs, dim=0).unsqueeze(0) # 1, S, C, H, W

            with torch.no_grad():
                
                trajs_e = run_model(model, rgbs, N)

            # store training_img and trajs_e
            rgb_array = rgbs[0][0].permute(1, 2, 0).numpy() # from tensor to numpy array
            training_img = cv2.resize(rgb_array , (360, 640)) # resizing image
            trajs_array = trajs_e.cpu().numpy()
            print(trajs_array[0][-1].shape)

            # define data
            if global_step ==1:
              a = np.array([(rgb_array, trajs_array[0][-1])])
            else:
              a=np.append(a, [(rgb_array, trajs_array[0][-1])],axis = 0)

        except FileNotFoundError as e:
            print('error', e)

    # save to npy file
    savez_compressed(video_name+'.npz', a)


In [ ]:
# downloading their reference model
!bash /content/drive/MyDrive/drive_folder/pips/get_reference_model.sh

In [8]:
model = create_model('reference_model')

reading ckpt from reference_model
...found checkpoint reference_model/model-000100000.pth


In [9]:
# unziping frames into data folder
!unzip /content/drive/MyDrive/frames_small.zip -d data_small

Archive:  /content/drive/MyDrive/frames_small.zip
replace data_small/content/frames/Tai chi56式夕陽美功夫扇-uOw-z7CR7x8/frames/frame_06296.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [50]:
# from tempfile import TemporaryFile
# create folder for numpy files


# for video_name in  os.listdir("data_small/content/frames"):
  # outfile = TemporaryFile() # create a np file
  # x = np.array() # create array

  # np.save(outfile, x) # save numpy file 

In [ ]:
generate_training_data(model,"Tai chiRen Guang Yi - Chen Style Taiji silk reeling part II-Wh3AmDsdQtM")

In [ ]:
from numpy import load

data = load('Tai chiRen Guang Yi - Chen Style Taiji silk reeling part II-Wh3AmDsdQtM.npz', allow_pickle=True)
lst = data.files
for item in lst:
  for x in range(len(item)):
    plt.imshow(data[item][x][0])
    plt.show()
    print(data[item][x][1])

In [ ]:
!pwd
# zip all numpy files
# cp NumPy zip file into drive